# Arquitectura RAG

In [1]:
%pip install faiss-cpu
%pip install langchain==0.1.16

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Usuario\Documents\GitHub\Arquitectura-RAG-Multimodal\.entorno_virtual\Scripts\python.exe -m pip install --upgrade pip' command.


  Attempting uninstall: packaging
    Found existing installation: packaging 26.0
    Uninstalling packaging-26.0:
      Successfully uninstalled packaging-26.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Usuario\Documents\GitHub\Arquitectura-RAG-Multimodal\.entorno_virtual\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import faiss
import numpy as np
import re

indice = faiss.IndexFlatL2(768)

# Reproducibilidad
np.random.seed(42)
context_data = np.random.random((100, 768)).astype('float32')

indice.add(context_data)

print(f"{indice.ntotal} vectores de contexto indexados")

100 vectores de contexto indexados


In [4]:
def busqueda_semantica(solicitud_incrustada, indexada, top_k=5):
    distances, indices = indexada.search(solicitud_incrustada, top_k)
    return indices

solicitud_incrustada = np.random.random((1, 768)).astype('float32')
indices_recuperados = busqueda_semantica(solicitud_incrustada, indice)
print(f"Indice recuperado: {indices_recuperados}")

Indice recuperado: [[26 38 11 78 12]]


In [5]:
%pip install transformers
%pip install torch

from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Usuario\Documents\GitHub\Arquitectura-RAG-Multimodal\.entorno_virtual\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\Usuario\Documents\GitHub\Arquitectura-RAG-Multimodal\.entorno_virtual\Scripts\python.exe -m pip install --upgrade pip' command.
c:\Users\Usuario\Documents\GitHub\Arquitectura-RAG-Multimodal\.entorno_virtual\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Usuario\Documents\GitHub\Arquitectura-RAG-Multimodal\.entorno_virtual\lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` enviro

In [ ]:
contexto_texto = [
    "La ISO/IEC 27001 es la norma más conocida del mundo para sistemas de gestión de la seguridad de la información (SGSI). Esta norma define los requisitos que debe cumplir un SGSI. "
    "La norma ISO/IEC 27001 proporciona a las empresas de cualquier tamaño y de todos los sectores orientaciones para establecer, implantar, mantener y mejorar de manera continua un sistema de gestión de la seguridad de la información. "
    "La conformidad con la ISO/IEC 27001 implica que una organización o empresa ha implantado un sistema para gestionar los riesgos relacionados con la seguridad de los datos que posee o maneja, y que este sistema respeta todas las buenas prácticas y principios contemplados en esta Norma Internacional."
]

prompt_template = PromptTemplate(
    "Utiliza solamente el siguiente contexto para responder la pregunta. Si no encuentras la respuesta en el contexto, di: No tengo sufienciente información."

    input_variables=["question", "context"],
    template="Question: {question}\nContext: {context}\nAnswer:"
    
)

In [ ]:
historico_conversacion = ConversationBufferMemory(
    memory_key="historial_chat", return_messages=False)

def chat(question):
    solicitud_incrustada = np.random.rand(1, 768).astype("float32")
    recuperar_indices = busqueda_semantica(solicitud_incrustada, indice)
    context_texts = [f"Documentos {i}" for i in recuperar_indices[0]]

    historial_chat = historico_conversacion.load_memory_variables({}).get("historial_chat", "")

    prompt = prompt_template.format(
        historial_chat=historial_chat,
        question=question,
        context="\n".join(context_texts)
    )

    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs,
        max_new_tokens=100,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response[len(prompt):].strip()
    response = re.sub(r"[\r\n]+", " ", response)

    historico_conversacion.chat_memory.add_user_message(question)
    historico_conversacion.chat_memory.add_ai_message(response)

    return response

In [ ]:
print(chat("¿Qué principios debe seguir una empresa para cumplir con las normativas ISO/IEC 270001?"))

¿Quando seguir una empresa para cumplir con las normativas ISO/IEC 270001? Context: Documentos 7 Documentos 55 Documentos 11 Documentos 93 Documentos 34 Answer: ¿Quando seguir una empresa para cumplir con las normativas ISO/IEC 270001? Context: Documentos 7 Documentos 55 Documentos 11 Documentos 93
